In [1]:
# CHE 597: Process Synthesis
# Course Code to try out various calculations

In [93]:
# Enthalpies of Formations kJ/mol
# Information from the NIST JANAF Thermochemical Tables
# janaf.nist.gov
# Other components from Engineering Toolbox

H_f0_CO_g = -110.527
H_f0_CO2_g = -393.522
H_f0_H2O_g = -241.826
H_f0_H2O_l = -285.830
H_f0_NH3_g = -45.898
H_f0_CH4_g = -74.873
H_f0_CH3OH_l = -239.2
H_f0_C2H6_g = -84
H_f0_C2H5OH_l = -277.6
H_f0_C3H8_g = -103.8
H_f0_C4H10_g = -125.7


In [94]:
# Gibbs Energies of Formations at standard temperature and pressure
G_f0_CO_g = -137.163
G_f0_CO2_g = -394.389
G_f0_H2O_g = -228.582
G_f0_H2O_l = -237.141
G_f0_NH3_g = -16.367
G_f0_CH4_g = -50.768
G_f0_CH3OH_l = -166.6
G_f0_C2H6_g = -32
G_f0_C2H5OH_l = -174.8
G_f0_C3H8_g = -23.4
G_f0_C4H10_g = -17


In [95]:
# Consider a methanol production process
# CH4_g + 1/2 O2 ==> CH3OH_l

# Overall Energy Balance
# \delta H = 
H_CH4_CH3OH = H_f0_CH3OH_l - H_f0_CH4_g
print(H_CH4_CH3OH)


-164.327


In [96]:
# For a methanol process with steam reforming
# CH4_g + H2O_l ==> CH3OH_l + H2_g

# Overall Energy Balance

H_CH4_H2O_CH3OH = H_f0_CH3OH_l - H_f0_CH4_g - H_f0_H2O_l
print(H_CH4_H2O_CH3OH)

121.50299999999999


In [97]:
# Ethanol Production

# 2CH4_g + H2O_l ==> C2H5OH_l + 5H2_g

# Overall Energy Balance

H_CH4_H2O_C2H5OH = H_f0_C2H5OH_l - 2*H_f0_CH4_g - H_f0_H2O_l
print(H_CH4_H2O_C2H5OH)

157.97599999999997


In [98]:
# The Inverted Ethanol Process to Hydrogen

# C2H5OH_l + 3H2O ==> 2CO2_g + H2_g

H_C2H5OH_H2O_CO2_H2 = 2*H_f0_CO2_g - H_f0_C2H5OH_l - 3*H_f0_H2O_l
print(H_C2H5OH_H2O_CO2_H2)

348.04600000000005


In [99]:
# Constrained Optimization

# Some example


In [100]:
import numpy as np
from scipy.optimize import minimize

# Defining our objective function from the carbon balance
def objective(X):
    a, b, c, d, e, f = X
    return b*H_f0_CH4_g + c*H_f0_H2O_l + e*H_f0_CO2_g - H_f0_CH3OH_l

# using the other equations as equality constraints
def eq1(X):
    a, b, c, d, e, f = X
    return 1 - a - b - e

def eq2(X):
    a, b, c, d, e, f = X
    return 4 - 4*b - 2*c - 2*f

def eq3(X):
    a, b, c, d, e, f = X
    return 1 - c - 2*d - 2*e

# Now defining the inequality constraints
#def ineq1(X):
#    a, b, c, d, e, f = X
#    return b*H_f0_CH4_g + c*H_f0_H2O_l + e*H_f0_CO2_g - H_f0_CH3OH_l

def ineq2(X):
    a, b, c, d, e, f = X
    return b*G_f0_CH4_g + c*G_f0_H2O_l + e*G_f0_CO2_g - G_f0_CH3OH_l

# Set the other inequalities as bounds
bounds_a = (0, 4)
bounds_b = (0, 4)
bounds_c = (-4, 4)
bounds_d = (0, 4)
bounds_e = (-0.001, 0.001)
bounds_f = (-8, 0)

bnds = (bounds_a, bounds_b, bounds_c, bounds_d, bounds_e, bounds_f)

# initial guesses to help the program find the solution
X0 = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

# Just to test
print(objective(X0))

88.35499999999996


In [101]:
# Creating the constraint list
constr1 = {'type': 'eq', 'fun': eq1}
constr2 = {'type': 'eq', 'fun': eq2}
constr3 = {'type': 'eq', 'fun': eq3}
constr4 = {'type': 'ineq', 'fun': ineq2}

constr = [constr1, constr2, constr3, constr4]

sol = minimize(objective, X0, bounds=bnds, constraints=constr)
print(sol)

     fun: 24.61723611308949
     jac: array([   0.        ,  -74.87299919, -285.82999802,    0.        ,
       -393.52199936,    0.        ])
 message: 'Optimization terminated successfully'
    nfev: 14
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 4.84894513e-10,  1.00100000e+00,  4.89901035e-01,  2.56049483e-01,
       -1.00000000e-03, -4.91901036e-01])


In [102]:
x_s = sol.x
print(ineq2(x_s))

-3.7990466239534726e-07


In [103]:
import numpy as np
from scipy.optimize import minimize

# Defining our objective function from the carbon balance
def objective(X):
    a, b, c, d, e, f = X
    return b*G_f0_CH4_g + c*G_f0_H2O_l + e*G_f0_CO2_g - G_f0_CH3OH_l

# using the other equations as equality constraints
def eq1(X):
    a, b, c, d, e, f = X
    return 1 - a - b - e

def eq2(X):
    a, b, c, d, e, f = X
    return 4 - 4*b - 2*c - 2*f

def eq3(X):
    a, b, c, d, e, f = X
    return 1 - c - 2*d - 2*e

# Now defining the inequality constraints
def ineq1(X):
    a, b, c, d, e, f = X
    return b*H_f0_CH4_g + c*H_f0_H2O_l + e*H_f0_CO2_g - H_f0_CH3OH_l

#def ineq2(X):
#    a, b, c, d, e, f = X
#    return b*G_f0_CH4_g + c*G_f0_H2O_l + e*G_f0_CO2_g - G_f0_CH3OH_l

# Set the other inequalities as bounds
bounds_a = (0, 4)
bounds_b = (0, 4)
bounds_c = (-4, 4)
bounds_d = (0, 4)
bounds_e = (-0.001, 0.001)
bounds_f = (-8, 0)

bnds = (bounds_a, bounds_b, bounds_c, bounds_d, bounds_e, bounds_f)

# initial guesses to help the program find the solution
X0 = [0.2, 0.2, 0.2, 0.2, 0.2, 0.2]

# Creating the constraint list
constr1 = {'type': 'eq', 'fun': eq1}
constr2 = {'type': 'eq', 'fun': eq2}
constr3 = {'type': 'eq', 'fun': eq3}
constr4 = {'type': 'ineq', 'fun': ineq1}

constr = [constr1, constr2, constr3, constr4]

sol = minimize(objective, X0, bounds=bnds, constraints=constr)
print(sol)


     fun: -24.316698529310855
     jac: array([   0.        ,  -50.76799965, -237.14099884,    0.        ,
       -394.38899994,   -0.        ])
 message: 'Optimization terminated successfully'
    nfev: 14
     nit: 6
    njev: 2
  status: 0
 success: True
       x: array([ 3.28987458e-01,  6.70012539e-01,  6.59974922e-01,  1.69012537e-01,
        1.00000000e-03, -2.78579715e-10])


In [104]:
x_s = sol.x
print(ineq1(x_s))

-2.75634241120315e-06
